# Molecular Quantum Mechanics (CB2070)
## Computer lab 5: Two-particle densities and electron correlation
---
Name: J.H. Andersen with stolen content from VT24 and VT25 student submissions

Date: July 2025

---

In [ ]:
import veloxchem as vlx

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.set_printoptions(precision=4, suppress=True, linewidth=132) # printout format of NumPy arrays
au_to_nm = 0.0529177 # length conversion factor
au_to_ev = 27.2114 # energy conversion factor

### Molecule specification

In [ ]:
mol_str = """
H     0.000000    0.000000   -0.370500
H     0.000000    0.000000    0.370500
"""
molecule = vlx.Molecule.read_str(mol_str, units='angstrom')

### SCF optimization

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
basis = vlx.MolecularBasis.read(molecule, 'sto-3g')
scf_results = scf_drv.compute(molecule, basis)

### 1. Cost-gain analysis of the bond formation

From the virial theorem we have that
$$2\left<T\right> = n\left<V\right>$$
where $n$ is the exponent of $r$ in the expression for the potential operator $\hat{V}$.

1. What is $n$ in our case?
2. What is the expression for the total average energy?
3. Use this expression and the virial theorem above to get the electron kinetic and potential energies for the isolated H atom. Use the value of `h_h` provided in the code below.

The electronic Hamiltonian:
$$\hat{H}_e = T_e + V_{en} + V_{ee} + V_{nn}$$

Where the kinetic operator is
$$\hat{T}_e = -\sum_{i=1}^{N}\frac{1}{2}\nabla^2_i$$
and the potential is 
$$\hat{V}_e = -\sum_{i=1}^{N}\sum_{A=1}^M\frac{Z_A}{r}$$

From the virial theorem we have that
$$2\left<T\right> = n\left<V\right>$$
where $n$ is the exponent of $r$ in the potential. In our case we have $n=-1$ and we get
$$2\left<T\right> = -\left<V\right>$$
Which means that twice the average total kinetic energy equals the negative average total potential energy. We also have that the average total energy is the sum of the kinetic and potential average total energies
$$\left<E\right> = \left<T\right> + \left<V\right>$$
We can then write
$$\left<E\right> = -\frac{1}{2}\left<V\right> + \left<V\right> \rightarrow \left<V\right> = 2\left<E\right>$$
Using this equation, we can determine the potential energy for the single electron in H atom, and then the kinetic energy as
$$\left<T\right> = -\frac{1}{2}\left<V\right> = -\left<E\right>$$




In [ ]:
# for the H2 molecule
nocc = molecule.number_of_electrons() // 2
norb = molecule.number_of_atoms()
print('Number of occupied MOs:', nocc)

C = scf_drv.scf_tensors['C_alpha']

# electron kinetic energy
kin_drv = vlx.KineticEnergyDriver()
T_ao = kin_drv.compute(molecule, basis).to_numpy()
T_mat = np.einsum('ai, ab, bj', C, T_ao, C)
T_e = np.einsum('ii', T_mat[:nocc, :nocc])

# electron-nuclear attraction
npot_drv = vlx.NuclearPotentialDriver()
V_ao = -npot_drv.compute(molecule, basis).to_numpy()
V_mat = np.einsum('ai, ab, bj', C, V_ao, C)
V_en = np.einsum('ii', V_mat[:nocc, :nocc])

# core Hamiltonian (one-electron)
h_h2 = T_e + V_en # for the H2 molecule
h_h = 0.5 * T_e + 0.25 * V_en # estimated core Hamiltonian for a single H atom

# nuclear-nuclear repulsion
V_nn = molecule.nuclear_repulsion_energy()

# electron-electron repulsion
fock_drv = vlx.FockDriver()
eri_ao = fock_drv.compute_eri(molecule, basis)
g_mat = np.einsum('abcd,ai,bj,ck,dl', eri_ao, C, C, C, C)
g = 2.0 * np.einsum('iijj', g_mat[:nocc, :nocc, :nocc, :nocc]) - np.einsum('ijji', g_mat[:nocc, :nocc, :nocc, :nocc])

# total energy of H2
e_h2 = 2.0 * h_h2 + g + V_nn # one-elec part is multiplierd by 2

In [ ]:
print('Energy of the H2 molecule: {:3.4f}'.format(e_h2))
print('One-electron part of the H2 energy: {:3.4f}'.format(h_h2))
print('Two-electron part of the H2 energy: {:3.4f}'.format(g))
print()

# the one-electron parts are per electron: in the equations for energy the terms are multiplied by 2 (number of electrons)
print('One-electron electron-nuclear potential energy of H2: {:3.4f}'.format(V_en))
print('One-electron kinetic energy of H2: {:3.4f}'.format(T_e))
print('One-electron part of the H2 energy: {:3.4f}'.format(h_h2))
print('Two-electron part of the H2 energy: {:3.4f}'.format(g))
print('Nuclear-nuclear repulsion energy: {:3.4f}'.format(V_nn))
print('Sum of all energy terms for H2: {:3.4f}'.format(e_h2))

In [ ]:
# using our derivations for the kinetic and potential energy for the single H atom:
T_e_h = -h_h
V_en_h = 2.0 * h_h
V_ee_h = 0.0 # only one electron, therefore zero elec-elec repulsion
V_nn_h = 0.0 # only one nucleus, therefore zero nuc-nuc repulsion

print('Estimated energy of an isolated H atom: {:3.4f}'.format(h_h))
print('Estimated potential energy of an isolated H atom: {:3.4f}'.format(V_en_h))
print('Estimated kinetic energy of an isolated H atom: {:3.4f}'.format(T_e_h))

In [ ]:
# calculating the cost/gain as difference between energies of the H2 molecule and single H atoms
print('Cost-gain analysis of bond formation in terms of kinetic energy: {:3.4f}'.format(T_e - T_e_h))
print('Cost-gain analysis of bond formation in terms of electron-nuclear attraction: {:3.4f}'.format(V_en - V_en_h))
print('Cost-gain analysis of bond formation in terms of electron-electron repulsion: {:3.4f}'.format(g - 2 * V_ee_h))
print('Cost-gain analysis of bond formation in terms of nuclear-nuclear repulsion: {:3.4f}'.format(V_nn - 2 * V_nn_h))
print()
print('Sum of all differences: {:3.4f}'.format(2*(T_e - T_e_h) + 2*(V_en - V_en_h) + (g - 2*V_ee_h) + (V_nn - 2*V_nn_h)))

By creating a bond between two H atoms, it will be at the cost of introducing elec-elec and nuc-nuc repulsion energies which are zero for the single atoms.
However, out calculations show that the increase in elec-nuc attraction energy compensates for the cost, resulting in an overall negative change in energy, meaning that the forming of a bond is favorable.

### 2. Explain the (lack of) coupling between blocks in the CISD Hamiltonian

Singly excited determinants have overall ungerade symmetry and the doubly excited determinant has overall gerade symmetry, considering the symmetry products of the occupied orbitals. The Hamiltonian has gerade symmetry, and thus the overall symmetry product of the matrix elements of singles and the double is ungerade, and thereby zero.

The HF ground state is $\ket{1 \bar{1}}$ and the double excited state is $\ket{2 \bar{2}}$. The single excited states are $\ket{2 \bar{1}}, \ket{\bar{2} \bar{1}}, \ket{1 2}, \ket{1 \bar{2}}$. The HF ground state has two electron in the gerade orbitals and is of g symmetry. The double excited state has two electrons in the ungerade orbitals and is also of g symmetry ("minus and minus gives plus"). However, all the single excited states have one electron in the gerade orbital and one in the ungerade orbital, which means that they have u symmetry. Since the single and double excited state do not have the same symmetry they do not couple. Furthermore, the symmetry of the exact ground-state wave function has g symmetry. Therefore, only determinants of g symmetry can appear in the expansion of the exact ground-state wave function, meaning that the CID provides a full CI description

### 3. Determine the CID state

With $|\Psi \rangle$ being a Slater determinant and $\hat{\Omega}$ a two-electron operator

$$
\hat{\Omega} = \sum_{j>i}^N \hat{\omega}(i,j)
$$

the following relations hold for the corresponding integrals

\begin{align*}
    \langle \Psi | \hat{\Omega} | \Psi \rangle &=
    \frac{1}{2} \sum_{i,j}^N
    \left[\rule{0pt}{12pt}
    \langle ij| \hat{\omega} | ij \rangle - \langle ij|  \hat{\omega} |ji \rangle
    \right]
    \\
    \langle \Psi | \hat{\Omega} | \Psi_{i}^{s} \rangle &=
    \sum_{j}^N
    \left[\rule{0pt}{12pt}
    \langle ij|  \hat{\omega} |sj \rangle - \langle ij|  \hat{\omega} |js \rangle
    \right]
    \\
    \langle \Psi | \hat{\Omega} | \Psi_{ij}^{st} \rangle &=
     \langle ij|  \hat{\omega} |st \rangle - \langle ij|  \hat{\omega} |ts \rangle
\end{align*}

Complete the formula
$$\mathbf{H}^{CID} = \begin{bmatrix} 2h_{11} + J_{11} & K_{12} \\
                                    K_{12} & 2h_{22} + J_{22} \end{bmatrix}$$

In [ ]:
# define the CID Hamiltonian using the one- and two-electron integrals determined in (1) above
H_CID = np.zeros((2,2))

H_CID[0,0] = 2 * (T_mat[0,0] + V_mat[0,0]) + g_mat[0,0,0,0]
H_CID[1,1] = 2 * (T_mat[1,1] + V_mat[1,1]) + g_mat[1,1,1,1]
H_CID[0,1] = g_mat[0,1,0,1]
H_CID[1,0] = g_mat[0,1,0,1]

print('CID Hamiltonian:\n', H_CID)

# diagonalize the CID Hamiltonian
E_CID, V_CID = np.linalg.eigh(H_CID)

# get the coefficients
c0 = V_CID[0,0]
c1 = V_CID[0,1]

### 4.  One-particle density

The CID state equals
$$
| \Psi \rangle =  c_0 | \Psi_\mathrm{HF} \rangle + c_1 | \Psi_{u\bar{u}} \rangle
$$

We have
$$
n(\mathbf{r}) =
\langle \Psi | \hat{n}(\mathbf{r}) | \Psi \rangle
$$
with
$$
\hat{n}(\mathbf{r}) = 
\sum_{i=1}^N
\delta(\mathbf{r} - \mathbf{r}_i)
$$

The general relations for matrix elements of a one-electron operator can now be used
\begin{align*}
    \langle \Psi | \hat{\Omega} | \Psi \rangle &=
   \sum_{i=1}^N
    \langle i| \hat{\omega} |i \rangle
    \\
    \langle \Psi | \hat{\Omega} | \Psi_{ij}^{st} \rangle &= 0
\end{align*}


#### Derive the formula for the one-particle density expressed in terms of spatial orbitals
Complete the equations
$$n^{HF}(\mathbf{r}) = 2|\sigma_g(\mathbf{r})|^2$$

$$n^{CID}(\mathbf{r}) = 2|c_0|^2|\sigma_g(\mathbf{r})|^2 + 2|c_1|^2|\sigma_u(\mathbf{r})|^2 $$


In [ ]:
# visualization driver
vis_drv = vlx.VisualizationDriver()

In [ ]:
# get the MOs along the z axis
mol_orbs = scf_results['C_alpha']

n = 100
coords = np.zeros((n,3))
coords[:,2] = np.linspace(-3, 3, n, endpoint=True)

sigma_g = np.array(vis_drv.get_mo(coords, molecule, basis, mol_orbs, 0))
sigma_u = np.array(vis_drv.get_mo(coords, molecule, basis, mol_orbs, 1))

# implement your formulas
n1_HF = 2 * sigma_g**2
n1_CID = 2 * c0**2 * sigma_g**2 + 2 * c1**2 * sigma_u**2

In [ ]:
fig = plt.figure(figsize = (8,6))

plt.title('One-particle densities')
plt.plot(coords[:,2], n1_HF, label='HF')
plt.plot(coords[:,2], n1_CID, label='CID')
plt.xlabel('Internuclear axis coordinate (Bohr)')
plt.ylabel('One-particle density (a.u.)')
plt.legend()

plt.show()

#### Discuss how the one-particle densities compare between HF and CID

The densities are very similar. This indicates that the CID correlation correction do not contribute with much extra information on the one-electron description compared to HF.

### 5. Two-particle density
The CID state equals
$$
| \Psi \rangle =  c_0 | \Psi_\mathrm{HF} \rangle + c_1 | \Psi_{g\bar{g}}^{u\bar{u}} \rangle
$$

We have
$$
n(\mathbf{r}, \mathbf{r}') =
\langle \Psi | \hat{n}(\mathbf{r}, \mathbf{r}') | \Psi \rangle
$$
with
$$
\hat{n}(\mathbf{r}, \mathbf{r}') = 
\sum_{j>i}^N \left[
\delta(\mathbf{r} - \mathbf{r}_i) \delta(\mathbf{r}' - \mathbf{r}_j)
+
\delta(\mathbf{r} - \mathbf{r}_j) \delta(\mathbf{r}' - \mathbf{r}_i)
\right]
$$

The general relations for matrix elements of a two-electron operator can now be used
\begin{align*}
    \langle \Psi | \hat{\Omega} | \Psi \rangle &=
    \frac{1}{2} \sum_{i,j}^N
    \Big[
    \langle ij| \hat{\omega} |ij \rangle - \langle ij | \hat{\omega} | ji \rangle
    \Big]
    \\
    \langle \Psi | \hat{\Omega} | \Psi_{ij}^{st} \rangle &=
     \langle ij| \hat{\omega} |st \rangle - \langle ij| \hat{\omega} |ts \rangle
\end{align*}


#### Derive the resulting formula for the two-particle density in terms of spatial orbitals
Complete the equations

$$n^{HF}(\mathbf{r}, \mathbf{r}') = 2 | \sigma_u (\mathbf{r}) |^2 | \sigma_u (\mathbf{r'}) |^2 $$

$$n^{CID}(\mathbf{r}, \mathbf{r}') = 2|c_0|^2|\sigma_g(\mathbf{r})|^2|\sigma_g(\mathbf{r}')|^2 + 2|c_1|^2|\sigma_u(\mathbf{r})|^2|\sigma_u(\mathbf{r}')|^2+2c_0c_1 \, \sigma_g^*(\mathbf{r})\sigma_g^*(\mathbf{r}')\sigma_u(\mathbf{r})\sigma_u(\mathbf{r}')$$


In [ ]:
# electron 1 at the position of the hydrogen nucleus
# electron 2 anywhere on the internuclear axis

Hz = -0.370500
h1 = [[0, 0, Hz]]

sigma_g_at_h1 = vis_drv.get_mo(h1, molecule, basis, mol_orbs, 0)[0]
sigma_u_at_h1 = vis_drv.get_mo(h1, molecule, basis, mol_orbs, 1)[0]

# implement your formulas
n12_HF = 2 * sigma_g_at_h1**2 * sigma_g**2
n12_CID = 2 * c0**2 * sigma_g_at_h1**2 * sigma_g**2 + 2 * c1**2 * sigma_u_at_h1**2 * sigma_u**2 + 4 * c0*c1 * sigma_g_at_h1 * sigma_g * sigma_u_at_h1 * sigma_u

In [ ]:
fig = plt.figure(figsize = (8,6))

plt.plot(coords[:,2], n12_HF)
plt.plot(coords[:,2], n12_CID)

plt.show()

#### Discuss how the two-particle densities compare between HF and CID

The two particle densities computed by HF and CID differs quite a lot. The HF two-particle density predicts equal probabilities of finding electron 2 at nucleus A and at nucleus B when electron 1 is found at nucleus A. This is highly unphysical as electrons repel each other by charge. The CID level of theory poses some correction to this as the probability of finding electron 2 at nucleus A is decreased when electron 1 is found at nucleus A, while the probability at nucleus B is increased.
The difference arise from the cross-term between the two determinants in $n(\mathbf{r},\mathbf{r}')$. 

# THE END